In [2]:
from glob import glob
import ctypes
import struct
import sys
from pathlib import Path

import numpy as np

import vtu
PyVtu=vtu.PyVtu
import small_functions
from ts_wrapper_Copy1 import TSWrapper

In [3]:
ts = TSWrapper()

In [4]:
vesicle = ts.parseDump('/mnt/c/Users/yoavr/Desktop/timestep_000087.vtu')

In [5]:
# import cffi # :(

In [6]:
path_to_trisurf = '/opt/workspace/msc_project/trisurf_samo/trisurf-ng'

In [7]:
ts = TSWrapper(path_to_trisurf)

In [8]:
vesicle = ts.parseDump('/opt/workspace/msc_project/simulations/QA_tests/cluster_version/feature_tests/test_wrapper_with_old_version/timestep_000000.vtu')

In [9]:
vesicle.contents.cm

In [10]:
tape=ts.parsetape(str(ts.path_to_trisurf/'src/tape'))

In [11]:
print(ts.pretty_text(tape))

    nshell 17
    ncxmax 100
    ncymax 100
    nczmax 100
    npoly 0
    nmono 20
    internal_poly 0
    nfil 0
    nfono 3
    R_nucleus 0
    R_nucleusX 0.0
    R_nucleusY 0.0
    R_nucleusZ 0.0
    pswitch 0
    constvolswitch 0
    constareaswitch 0
    stretchswitch 0
    xkA0 1.0
    constvolprecision 1e-14
    multiprocessing <ctypes.LP_c_char object at 0x7f58833ecdc0>
    brezveze0 2
    brezveze1 50
    brezveze2 50
    xk0 10.0
    dmax 1.7
    dmin_interspecies 1.2
    stepsize 0.15
    kspring 800.0
    xi 100.0
    pressure 10000.0
    iterations 100
    inititer 0
    mcsweeps 200
    quiet 0
    shc 21
    number_of_vertices_with_c0 100
    c0 0.5
    w 10.0
    F 2.0
    plane_confinement_switch 0
    plane_d 10.0
    plane_F 10.0


In [12]:
[vesicle.contents.vlist.contents.vtx[i].contents.c for i in range(50,100)]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0]

In [13]:
vtx=vesicle.contents.vlist.contents.vtx[97]
vtx_old=ts.ts_vertex(x=0,y=0,z=0)

In [25]:
vtx.contents.z

7.276000458314542

In [19]:
import ctypes

In [21]:
ctypes.pointer(vtx_old)

In [26]:
ts.ts.direct_force_energy.restype=ctypes.c_double
ts.ts.direct_force_energy(vesicle, vtx, ctypes.pointer(vtx_old))

-17.83702389279913

In [44]:
def parse_line_type(line,types):
    for t in types:
        if line.startswith(t):
            _, left, right = line.partition(t)
    left, right = left.strip(), right.strip()
    while left.endswith('*'):
        left = left[:-1].strip()
        right = '*' + right
    return left, right


In [60]:
int.from_bytes(vesicle.contents.tape.contents.force_model,sys.byteorder)

16

In [61]:
ts.ts.direct_force_energy.restype=ctypes.c_double
ts.ts.direct_force_energy(vesicle,ctypes.pointer(vtx),ctypes.pointer(vtx_old))


-1.0

In [19]:
defines = []
skip=False
for k,v in ts.items[1:]:
    if k=='#':
        if skip:
            skip=not v[0].startswith('#endif')
            continue
        if v[0].startswith('#define'):
            defines.append((k,v[0]))
        if v[0].startswith('#if'):
            skip=True

In [20]:
defines

[('#', '#define _GENERAL_H'),
 ('#', '#define TS_SUCCESS 0'),
 ('#', '#define TS_FAIL 1'),
 ('#', '#define TS_ID_FILAMENT 1'),
 ('#', '#define TS_DOUBLE_DOUBLE'),
 ('#', '#define TS_COORD_CARTESIAN 0'),
 ('#', '#define TS_COORD_SPHERICAL 1'),
 ('#', '#define TS_COORD_CYLINDRICAL 2'),
 ('#', '#define VTX(n) &(vlist->vtx[n])'),
 ('#', '#define VTX_DATA(n) vlist->vtx[n].data'),
 ('#', '#define CPF_CLOEXEC 1')]

In [5]:
global_vars = {}
for item in filter(lambda x:x[0]=='extern',ts.items):
    pre,most,post = ts.text_bracket_partition("\n".join(item[1]))
    defline = ' '.join((pre,post)).strip() # typedef [type...] name
    ty = ' '.join(defline.split()[1:-1]) if 'inline' not in defline else 'inline'
    name = defline.split()[-1].strip(' ;')
    if ty!='inline':
        global_vars[name]=ts.ts_types[ty].in_dll(ts.ts,name)

In [8]:
global_vars['command_line_args'].dump_fullfilename

b'dump.bin'

In [9]:
vtx = ts.ts_types['ts_vertex']()

In [74]:
def clean_text_comments(text):
    mid="/*"
    text = text.replace('\t','    ')
    while mid:
        pre,mid,post = text.partition('/*')
        text = pre+post.partition('*/')[-1]
    return '\n'.join(y for x in text.splitlines() if (y:=x.partition('//')[0]).strip())

In [75]:
print(clean_text_comments(text))

In [76]:
def parse_line_type_inverse(line,types):
    left, _, right = line.rpartition(' ')
    for t in types:
        if t+' ' in line or t+'*' in line:
            _, left, right = line.partition(t)
            break
    return left.strip(), right.strip()

In [83]:
text='long int* my_func(int* j, ts_poly_list **l);'
for file in ts.path_to_trisurf.glob("./src/*.h"):
    with open(file,'r') as f:
        text=text+'\n'+f.read()
funcs = {}
mid = "("
text = clean_text_comments(text)
while mid:
    pre,mid,post = text.partition('(')
    args, mid2, post = post.partition(')')
    opts=[]
    last_2 = pre.splitlines()[-1]
    if len(last_2)>2 and '#' not in last_2[0]:
        ty, name = parse_line_type_inverse(last_2,ts.ts_types)
        while name.startswith('*'):
            opts.append(ctypes.POINTER)
            name=name[1:].strip()
        ty = ts.ts_types.get(ty)
        for opt in opts:
            ty = opt(ty)
        funcs[name]=(ty,args)
    text=post
    

In [ ]:
funcs.values()

In [86]:
func_args = {}
for func, (_, args) in funcs.items():
    new_args=[]
    for arg in args.split(','):
        opts=[]
        ty, name = parse_line_type_inverse(arg,ts.ts_types)
        while name.startswith('*'):
            opts.append(ctypes.POINTER)
            name=name[1:].strip()
        ty = ts.ts_types.get(ty)
        for opt in opts:
            ty = opt(ty)
        new_args.append( (name,ty))
    func_args[func]=(new_args)

In [96]:
prox = lambda x: x.replace('*',' * ').split(',')
[prox(x) for x in ('int a','long unsigned*j[];', 'int long,*j[],* *q;')]

[['int a'], ['long unsigned * j[];'], ['int long', ' * j[]', ' *   * q;']]

In [173]:
def parse_type(line,types):
    """Parse a general type declaration line"""
    first, *rest = line.replace('*',' * ').strip(' ;').split(',')
    base_type = first.replace('*','').replace('[','').replace(']','').replace(';','').rpartition(' ')[0].strip()
    left = first.replace(base_type,'').strip()
    return base_type, [left, *rest]
    

In [174]:
[parse_type(x,ts.ts_types) for x in ('int a','long unsigned*j[];', 'int long,*j[],* *q;','ts_list * b, q[],b')]

[('int', ['a']),
 ('long unsigned', ['* j[]']),
 ('int', ['long', ' * j[]', ' *   * q']),
 ('ts_list', ['*  b', ' q[]', 'b'])]

In [87]:
func_args

{'my_func': [('j', __main__.LP_c_int), ('l', __main__.LP_LP_ts_poly_list)],
 'mean_curvature_and_energy': [('vesicle', ts_wrapper_Copy1.LP_ts_vesicle)],
 'energy_vertex': [('vtx', ts_wrapper_Copy1.LP_ts_vertex)],
 'bond_energy': [('bond', ts_wrapper_Copy1.LP_ts_bond),
  ('poly', ts_wrapper_Copy1.LP_ts_poly)],
 'sweep_attraction_bond_energy': [('vesicle', ts_wrapper_Copy1.LP_ts_vesicle)],
 'attraction_bond_energy': [('bond', ts_wrapper_Copy1.LP_ts_bond),
  ('w', ctypes.c_double)],
 'direct_force_energy': [('vesicle', ts_wrapper_Copy1.LP_ts_vesicle),
  ('vtx', ts_wrapper_Copy1.LP_ts_vertex),
  ('vtx_old', ts_wrapper_Copy1.LP_ts_vertex)],
 'stretchenergy': [('vesicle', ts_wrapper_Copy1.LP_ts_vesicle),
  ('triangle', ts_wrapper_Copy1.LP_ts_triangle)],
 'parseDump': [('dumpfname', ctypes.LP_c_char)],
 'parseTrisurfTag': [('doc', None), ('cur', None)],
 'setGlobalTapeTXTfromTapeTag': [('doc', None), ('cur', None)],
 'parseTrisurfVtxn': [('vlist', ts_wrapper_Copy1.LP_ts_vertex_list),
  ('doc'

In [31]:
import ts_wrapper_Copy1 as ts2

In [46]:
funcs

{'sweep_vertex_curvature_energy': (ctypes.c_char, 'ts_vesicle *vesicle'),
 'sweep_vertex_forces': (ctypes.c_char, 'ts_vesicle *vesicle'),
 'laplace_beltrami_curvature_energy': (ctypes.c_char,
  'ts_vesicle *vesicle, ts_vertex *vtx'),
 'tensor_curvature_energy': (ctypes.c_char,
  'ts_vesicle *vesicle, ts_vertex *vtx'),
 'vertex_curvature_energy': (ctypes.c_char,
  'ts_vesicle *vesicle, ts_vertex *vtx'),
 'bond_energy': (ctypes.c_char, 'ts_bond *bond,ts_poly *poly'),
 'sweep_attraction_bond_energy': (ctypes.c_char, 'ts_vesicle *vesicle'),
 'attraction_bond_energy': (ctypes.c_char,
  'ts_vesicle *vesicle, ts_bond *bond'),
 'direct_force_energy': (ctypes.c_double,
  'ts_vesicle *vesicle, ts_vertex *vtx, ts_vertex *vtx_old'),
 'direct_force_from_Fz_balance': (ctypes.c_double,
  'ts_vesicle *vesicle, ts_vertex *vtx, ts_vertex *vtx_old'),
 'total_force_on_vesicle': (ctypes.c_char, 'ts_vesicle *vesicle'),
 'adhesion_energy_diff': (ctypes.c_double,
  'ts_vesicle *vesicle, ts_vertex *vtx, ts_ver

In [76]:
list(ts.path_to_trisurf.glob("./src/*.h"))

[PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/energy.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/restore.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/poly.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/constvol.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/bond.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/vesicle.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/sh.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/shcomplex.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/bondflip.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/cell.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/timestep.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/general.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/vertexmove.h'),
 PosixPath('/opt/workspace/msc_project/cluster-trisurf/src/cluster.h'),

In [3]:
file = Path("/mnt/c/Users/yoavr/Desktop/timestep_000087.vtu")

In [4]:
v = PyVtu(file)

In [5]:
vesicle = ts.parseDump(str(file))

In [6]:
vertices = [
vesicle.contents.vlist.contents.vtx[i] for i in range(vesicle.contents.vlist.contents.n)]

In [7]:
vertices[2]

In [8]:
dist = [
ts.adhesion_geometry_distance(vesicle, vtx) for vtx in vertices]

In [9]:
int.from_bytes(vesicle.contents.tape.contents.adhesion_geometry,sys.byteorder)

2

In [10]:
vesicle.contents.tape.contents.adhesion_radius

10.0

In [11]:
np.array(dist)[np.array(dist)<1]

array([0.91339441, 0.96547936, 0.91553219, 0.89688824, 0.86787479])

In [12]:
dist2=(((v.pos - [[0,0,-20]])**2).sum(axis=1)**0.5-10)

In [13]:
dist=np.array(dist)

In [14]:
i=568
dist2[i],v.pos[i],dist[i],(vertices[i].contents.x,vertices[i].contents.y,vertices[i].contents.z)

(0.9133944089548862,
 array([-0.69210235, -2.19473144, -9.33199523]),
 0.9133944089548862,
 (-0.6921023463065015, -2.1947314360371157, -9.331995229129523))

In [15]:
v.type[i]

4

In [16]:
int.from_bytes(vertices[i].contents.type,sys.byteorder) & ts.enum_values['is_adhesive_vtx']

4

In [17]:
ts.adhesion_energy_diff(vesicle, vertices[i],vertices[0])

-1.25

In [21]:
ts.adhesion_geometry_side(vesicle, vertices[i])

b'\x01'

In [90]:
vesicle.contents.tape.contents.adhesion_model==ts.enum_values['adhesion_step_potential']

False

In [3]:
max_cluster_size = 450
cluster_hist = np.zeros(max_cluster_size)

In [4]:
file_regex = ("/mnt/c/Users/yoavr/Desktop/paraview_pipeline"
                "/hello_chemfarm/anisotropy/5_from_mitja/x0000_b0k0"
                "/time*")

file_regex = ("/opt/workspace/msc_project/simulations"
                "/QA_tests/throwaway"
                "/time*")

files = sorted(glob(file_regex))
files[:6]

['/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000000.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000001.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000002.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000003.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000004.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000005.vtu']

In [5]:
for file in files[:3]:
    vesicle=ts.parseDump(file)
    cstlist=ts.init_cluster_list()
    ts.clusterize_vesicle(vesicle, cstlist)    
    for j in range(cstlist.contents.n):
        size=cstlist.contents.cluster[j].contents.nvtx
        cluster_hist[size-1]+=1.0000

In [12]:
v=PyVtu(files[2])

In [17]:
v.pos[500], vtx_500.contents.x,vtx_500.contents.y,vtx_500.contents.z

(array([ 0.05522578,  0.04100399, -6.23498369]),
 0.05522578450024107,
 0.04100399122861435,
 -6.234983690768586)

In [18]:
ts.adhesion_geometry_distance(vesicle, vesicle.contents.vlist.contents.vtx[500])

0.0

In [12]:
v=vtu.PyVtu(file)

In [13]:
for field, ty in vesicle.contents._fields_:
    if (ty in {ctypes.c_int, ctypes.c_uint, ctypes.c_double}):
        print(field,vesicle.contents.__getattribute__(field))
    elif (ty in {ctypes.c_double*3,}):
        print(field, vesicle.contents.__getattribute__(field)[:])
    else:
        print(field,vesicle.contents.__getattribute__(field))

dmax 2.8899999999999997
stepsize 0.15
cm [0.0, 0.0, 0.0]
fx 0.0
fy 0.0
fz 0.0
volume 0.0
area 0.0
spring_constant 800.0
pressure 0.0
R_nucleus 0.0
R_nucleusX 0.0
R_nucleusY 0.0
R_nucleusZ 0.0
nucleus_center [0.0, 0.0, 0.0]
tape <ts_wrapper_Copy1.LP_ts_tape object at 0x7f7fe5adcec0>
sphHarmonics <ts_wrapper_Copy1.LP_ts_spharm object at 0x7f7fe5adcec0>
poly_list <ts_wrapper_Copy1.LP_ts_poly_list object at 0x7f7fe5adcec0>
filament_list <ts_wrapper_Copy1.LP_ts_poly_list object at 0x7f7fe5adcec0>
vlist <ts_wrapper_Copy1.LP_ts_vertex_list object at 0x7f7fe5adcec0>
blist <ts_wrapper_Copy1.LP_ts_bond_list object at 0x7f7fe5adcec0>
tlist <ts_wrapper_Copy1.LP_ts_triangle_list object at 0x7f7fe5adcec0>
clist <ts_wrapper_Copy1.LP_ts_cell_list object at 0x7f7fe5adcec0>
confinement_plane <ts_wrapper_Copy1.ts_confinement_plane object at 0x7f7fe5adcec0>
nshell 10


In [15]:
tape_opts = small_functions.extract_tape_options(v.tape)
for i, (field, ty) in enumerate(vesicle.contents.tape.contents._fields_):
    attrib = vesicle.contents.tape.contents.__getattribute__(field)
    if (ty in {ctypes.c_int, ctypes.c_uint, ctypes.c_double,ctypes.c_ulong}):
        val = attrib
    elif (ty in {ctypes.c_double*3,}):
        val = attrib[:]
    elif (ty in {ctypes.c_char,ctypes.c_byte}):
        val = int.from_bytes(attrib,sys.byteorder)
    else:
        val=attrib
    print(i,field,val,tape_opts.get(field))

0 tape_text <ctypes.LP_c_char object at 0x7f7fe5adcf40> None
1 R_nucleus 0.0 0
2 R_nucleusX 0.0 0
3 R_nucleusY 0.0 0
4 R_nucleusZ 0.0 0
5 xkA0 1.0 1.0
6 xkV0 1.0 1.0
7 V0 0.0 0
8 A0 0.0 0
9 Vfraction 1.0 None
10 constvolprecision 1e-14 1e-14
11 xk0 20.0 20
12 xk2 -20.0 -20
13 dmax 1.7 1.7
14 dmin_interspecies 1.2 1.2
15 stepsize 0.15 0.15
16 kspring 800.0 None
17 xi 100.0 100
18 pressure 0.0 0.0
19 c0 0.5 0.5
20 d0 0.5 0.5
21 w 1.0 1.0
22 F 1.0 1.0
23 plane_d 10.0 10
24 plane_F 1.0 1.0
25 vicsek_strength 0.1 0.1
26 vicsek_radius 4.0 4.0
27 adhesion_z -5.0 -5
28 adhesion_cutoff 1.0 1
29 adhesion_strength 1.0 1.0
30 adhesion_radius 5.0 5.0
31 adhesion_scale 5.0 5.0
32 adhesion_factor 2.0 2.0
33 min_dihedral_angle_cosine 0.1 0.1
34 mcsweeps 1000 1000
35 random_seed 1682609434 1682609434
36 iterations 10 10
37 inititer 0 0
38 nshell 10 10
39 ncxmax 100 None
40 ncymax 100 None
41 nczmax 100 None
42 number_of_vertices_with_c0 50 50
43 npoly 0 0
44 nmono 20 20
45 internal_poly 0 0
46 nfil 0 0

In [10]:
int.from_bytes(vesicle.contents.tape.contents.curvature_model,sys.byteorder)

15

In [ ]:
small_functions.extract_tape_options(v.tape)

In [21]:
name,val='b=9'.split('=')
name,int(val)

('b', 9)

In [30]:
ee = {name:{k: int(v) for k,v in map(lambda x: x.split('='),val)} for name,val in ts.enums.items()}

In [36]:
{k:v for d in ee.values() for k,v in d.items()}

{'is_bonding_vtx': 1,
 'is_active_vtx': 2,
 'is_adhesive_vtx': 4,
 'is_anisotropic_vtx': 8,
 'is_reserved_0_vtx': 16,
 'is_vicsek_vtx': 32,
 'is_edge_vtx': 64,
 'is_ghost_vtx': -128,
 'is_bonding_type_specific': 1,
 'is_anisotropic_bonding_nematic': 2,
 'to_disable_calculate_laplace_beltrami': 64,
 'to_calculate_sum_angle': 1,
 'to_calculate_shape_operator': 2,
 'to_update_director_shapeless': 4,
 'to_use_shape_operator_energy': 8,
 'to_use_shape_for_anisotropy_only': 16,
 'to_not_rotate_directors': 32,
 'to_use_sum_angle_for_kx2_only': 128,
 'model_laplace_beltrami_only': 0,
 'model_isotropic': 1,
 'model_shape_operator_only': 74,
 'model_debug_old_energy': 7,
 'model_debug_new_energy': 15,
 'model_debug_parallerl_transport_directors': 35,
 'model_debug_assumed_final': 146,
 'model_vertex_meanW': 0,
 'model_active_neigh_interfer': 1,
 'model_concave_neigh_interfer': 2,
 'model_concave_neigh_disable': 3,
 'is_vicsek_model': 16,
 'model_vicsek_1overR': 17,
 'adhesion_step_potential': 1,

In [ ]:
raise RuntimeError("Ts test ahead")

#### Test the automatic wrapper

In [3]:
loc_trisurf = Path("/opt/workspace/msc_project/cluster-trisurf/")

# try: No need, this is a jupyter
with open(loc_trisurf/Path("src/general.h"),'r') as f:
    text = f.read()

In [168]:
items = []
curr = []
open_char = ''
comm_char = ''
for i, line in enumerate(text.splitlines()):
    print(line)
    line = line.partition('//')[0]
    line, comm, end = line.partition('/*')
    if comm:
        comm_char='/'
    if comm_char=='/':
        if '*/' in end:
            comm_char=''
        elif '*/' in line:
            comm_char=''
            line = line.partition('*/')[-1]
        else:
            continue
    line=line.strip()
    print('>',line)
    if open_char=='{':
        curr.append(line)
    if line.startswith("extern"):
        curr=[line]
        items.append(('extern',curr))
    if line.startswith("#"):
        curr=[line]
        items.append(('#',curr))
    if "enum" in line:
        curr=[line]
        items.append(('enum',curr))
    if 'typedef struct' in line:
        curr=[line]
        items.append(('typedef',curr))
        items.append(('struct',curr))
    elif 'typedef' in line:
        curr = [line]
        items.append(('typedef',curr))
    elif 'struct' in line and not open_char:
        curr=[line]
        items.append(('struct',curr))
    if '{' in line:
        open_char='{'
    if '}' in  line:
        open_char=''
    if items:
        print('+++',open_char,comm_char,len(items) ,'+++',items[-1])

/* vim: set ts=4 sts=4 sw=4 noet : */
> 
#ifndef _GENERAL_H
> #ifndef _GENERAL_H
+++   1 +++ ('#', ['#ifndef _GENERAL_H'])
#define _GENERAL_H
> #define _GENERAL_H
+++   2 +++ ('#', ['#define _GENERAL_H'])

> 
+++   2 +++ ('#', ['#define _GENERAL_H'])
#include <stdio.h>
> #include <stdio.h>
+++   3 +++ ('#', ['#include <stdio.h>'])
#include <stdarg.h>
> #include <stdarg.h>
+++   4 +++ ('#', ['#include <stdarg.h>'])
#include <gsl/gsl_complex.h>
> #include <gsl/gsl_complex.h>
+++   5 +++ ('#', ['#include <gsl/gsl_complex.h>'])
#include <math.h> // added #ifndef M_PI to stop vscode from complaning about undefined M_PI, but this should load it anyway and avert the ifndef
> #include <math.h>
+++   6 +++ ('#', ['#include <math.h>'])

> 
+++   6 +++ ('#', ['#include <math.h>'])
/** @brief This is a header file, defining general constants and structures.
  * @file header.h
  * @author Samo Penic
  * @date 5.3.2001
  * 
  * Header file for general inclusion in all the code, defining data structu

In [183]:
items[20:30]

[('typedef', ['typedef signed int ts_idx;']),
 ('typedef', ['typedef signed long ts_massive_idx;']),
 ('extern',
  ['extern inline ts_small_idx next_small(ts_small_idx i, ts_small_idx max);']),
 ('extern', ['extern inline ts_idx next_idx(ts_idx i, ts_idx max);']),
 ('extern',
  ['extern inline ts_small_idx prev_small(ts_small_idx i, ts_small_idx max);']),
 ('extern', ['extern inline ts_idx prev_idx(ts_idx i, ts_idx max);']),
 ('typedef', ['typedef unsigned int ts_cell_idx;']),
 ('typedef', ['typedef char ts_flag;']),
 ('typedef', ['typedef int ts_big_flag;']),
 ('enum',
  ['enum vertex_type {',
   'is_bonding_vtx=1,',
   'is_active_vtx=2,',
   'is_adhesive_vtx=4,',
   'is_anisotropic_vtx=8,',
   'is_reserved_0_vtx=16,',
   'is_vicsek_vtx=32,',
   'is_edge_vtx=64,',
   'is_ghost_vtx=-128,',
   '};'])]

In [172]:
defines = []
skip=False
for k,v in items[1:]:
    if k!='#':
        continue
    if skip:
        print(v,' skipped')
        if v[0].startswith('#endif'):
            skip=False
        continue
    print(k,v,end='')
    if v[0].startswith('#define'):
        defines.append((k,v[0]))
        print('  >>>  appended',end='')
    if v[0].startswith('#if'):
        skip=True
    print()
print('---')
defines

# ['#define _GENERAL_H']  >>>  appended
# ['#include <stdio.h>']
# ['#include <stdarg.h>']
# ['#include <gsl/gsl_complex.h>']
# ['#include <math.h>']
# ['#define TS_SUCCESS 0']  >>>  appended
# ['#define TS_FAIL 1']  >>>  appended
# ['#define TS_ID_FILAMENT 1']  >>>  appended
# ['#define TS_DOUBLE_DOUBLE']  >>>  appended
# ['#define TS_COORD_CARTESIAN 0']  >>>  appended
# ['#define TS_COORD_SPHERICAL 1']  >>>  appended
# ['#define TS_COORD_CYLINDRICAL 2']  >>>  appended
# ['#define VTX(n) &(vlist->vtx[n])']  >>>  appended
# ['#define VTX_DATA(n) vlist->vtx[n].data']  >>>  appended
# ['#define CPF_CLOEXEC 1']  >>>  appended
# ['#ifndef TS_VERSION']
['#define TS_VERSION "whatever"']  skipped
['#endif']  skipped
# ['#ifndef M_PI']
['#define M_PI 3.14159265358979323846']  skipped
['#endif']  skipped
# ['#endif']
---


[('#', '#define _GENERAL_H'),
 ('#', '#define TS_SUCCESS 0'),
 ('#', '#define TS_FAIL 1'),
 ('#', '#define TS_ID_FILAMENT 1'),
 ('#', '#define TS_DOUBLE_DOUBLE'),
 ('#', '#define TS_COORD_CARTESIAN 0'),
 ('#', '#define TS_COORD_SPHERICAL 1'),
 ('#', '#define TS_COORD_CYLINDRICAL 2'),
 ('#', '#define VTX(n) &(vlist->vtx[n])'),
 ('#', '#define VTX_DATA(n) vlist->vtx[n].data'),
 ('#', '#define CPF_CLOEXEC 1')]

In [239]:
def text_bracket_partition(text,sep=';'):
    pre, _, most = text.partition('{')
    pre, most = pre.strip(), most.strip()
    most,_,post = most.rpartition('}')
    most = most.split(sep)
    most = [y for x in most if (y := x.strip(' ;{}\n'))]
    post = post.strip(' \n;')
    return pre,most,post

for item in items:
    print(text_bracket_partition('\n'.join(item[1]),',' if item[0]=='enum' else ';')[1])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['is_bonding_vtx=1', 'is_active_vtx=2', 'is_adhesive_vtx=4', 'is_anisotropic_vtx=8', 'is_reserved_0_vtx=16', 'is_vicsek_vtx=32', 'is_edge_vtx=64', 'is_ghost_vtx=-128']
['is_bonding_type_specific=1', 'is_anisotropic_bonding_nematic=2']
['to_disable_calculate_laplace_beltrami=64', 'to_calculate_sum_angle=1', 'to_calculate_shape_operator=2', 'to_update_director_shapeless=4', 'to_use_shape_operator_energy=8', 'to_use_shape_for_anisotropy_only=16', 'to_not_rotate_directors=32', 'to_use_sum_angle_for_kx2_only=128', 'model_laplace_beltrami_only=0', 'model_isotropic=1', 'model_shape_operator_only=74', 'model_debug_old_energy=7', 'model_debug_new_energy=15', 'model_debug_parallerl_transport_directors=35', 'model_debug_assumed_final=146']
['model_vertex_meanW=0', 'model_active_neigh_interfer=1', 'model_concave_neigh_interfer=2', 'model_concave_neigh_disable=3', 'is_vicsek_model=16', 'model_vicsek_1overR=17']
[

In [228]:
enums = {}

for k,v in items[1:]:
    if k!='enum':
        continue
    retext = '\n'.join(v)
    pre,most,post=text_bracket_partition(retext,',')
    enums[pre.rpartition('enum')[-1].strip()+post]=most
enums

{'vertex_type': ['is_bonding_vtx=1',
  'is_active_vtx=2',
  'is_adhesive_vtx=4',
  'is_anisotropic_vtx=8',
  'is_reserved_0_vtx=16',
  'is_vicsek_vtx=32',
  'is_edge_vtx=64',
  'is_ghost_vtx=-128'],
 'bond_model_type': ['is_bonding_type_specific=1',
  'is_anisotropic_bonding_nematic=2'],
 'curvature_model_type': ['to_disable_calculate_laplace_beltrami=64',
  'to_calculate_sum_angle=1',
  'to_calculate_shape_operator=2',
  'to_update_director_shapeless=4',
  'to_use_shape_operator_energy=8',
  'to_use_shape_for_anisotropy_only=16',
  'to_not_rotate_directors=32',
  'to_use_sum_angle_for_kx2_only=128',
  'model_laplace_beltrami_only=0',
  'model_isotropic=1',
  'model_shape_operator_only=74',
  'model_debug_old_energy=7',
  'model_debug_new_energy=15',
  'model_debug_parallerl_transport_directors=35',
  'model_debug_assumed_final=146'],
 'force_model_type': ['model_vertex_meanW=0',
  'model_active_neigh_interfer=1',
  'model_concave_neigh_interfer=2',
  'model_concave_neigh_disable=3',
 

In [256]:
typedefs = {}

for item in items:
    if item[0]=='typedef':
        pre,_,post = text_bracket_partition('\n'.join(item[1]))
        defline = ' '.join((pre,post)).strip() # typedef [type...] name
        name = defline.split()[-1].strip(' ;')
        ty = ' '.join(defline.split()[1:-1]) if 'struct' not in ty else 'struct'
        typedefs2[name]=ty
typedefs

{'ts_int': 'int',
 'ts_uint': 'unsigned int',
 'ts_long': 'long',
 'ts_ulong': 'unsigned long',
 'ts_float': 'float',
 'ts_double': 'double',
 'ts_char': 'char',
 'ts_uchar': 'unsigned char',
 'ts_bool': 'char',
 'ts_small_idx': 'signed char',
 'ts_idx': 'signed int',
 'ts_massive_idx': 'signed long',
 'ts_cell_idx': 'unsigned int',
 'ts_flag': 'char',
 'ts_big_flag': 'int',
 'ts_coord': 'struct',
 'ts_vertex': 'struct ts_vertex',
 'ts_vertex_list': 'struct',
 'ts_bond': 'struct ts_bond',
 'ts_bond_list': 'struct ts_bond_list',
 'ts_triangle': 'struct ts_triangle',
 'ts_triangle_list': 'struct ts_triangle_list',
 'ts_cell': 'struct ts_cell',
 'ts_cell_list': 'struct ts_cell_list',
 'ts_spharm': 'struct',
 'ts_poly': 'struct ts_poly',
 'ts_poly_list': 'struct ts_poly_list',
 'ts_confinement_plane': 'struct',
 'ts_tape': 'struct',
 'ts_vesicle': 'struct',
 'ts_cluster': 'struct ts_cluster',
 'ts_cluster_list': 'struct',
 'ts_seen_vertex': 'struct',
 'ts_args': 'struct'}

In [253]:
typedefs = {}

for item in items:
    if item[0]=='typedef':
        name = item[1][-1].split()[-1].strip(' {};:/')
        ty = ' '.join(item[1][0].split()[1:]).strip(' {};:/')
        ty = 'struct' if 'struct' in ty else ' '.join(ty.split()[:-1])
        typedefs[name]=ty


In [255]:
typedefs==typedefs2

False

In [254]:
base_types = {
    'int': ctypes.c_int,
    'signed int': ctypes.c_int,
    'unsigned int': ctypes.c_uint,
    'long': ctypes.c_long,
    'signed long': ctypes.c_long,
    'unsigned long': ctypes.c_ulong,
    'float': ctypes.c_float,
    'double': ctypes.c_double,
    'char' : ctypes.c_char,
    'signed char' : ctypes.c_byte,
    'unsigned char': ctypes.c_ubyte,
}

In [126]:
ts_types={k:base_types.get(v) for k,v in typedefs.items()}|base_types

In [16]:
ts_types

NameError: name 'ts_types' is not defined

In [272]:
for name, ty in ts_types.items():
    if ty is None:
        ts_types[name] = type(name, (ctypes.Structure,),{"_fields_":[]})

In [362]:
def parse_struct(text,types):
    pre, most, post = text_bracket_partition(text)
    name = ' '.join([pre,post]).strip().split()[-1].strip(' ;')
    fields = []
    for field in most:
        if 'struct' in field:
            field = field.replace('struct','').strip()
        if not field:
            continue
        left, _, right = field.partition(' ')
        left, right = left.strip(), right.strip()
        while left.endswith('*'):
            left = left[:-1].strip()
            right = '*' + right
        for b in right.split(','):
            b=b.strip()
            ty=[]
            while b.startswith('*'):
                b=b[1:].strip()
                ty.append(ctypes.POINTER)
            while b.endswith(']'):
                b, _, d = b[:-1].rpartition('[')
                b,d = b.strip(),d.strip()
                ty.append(lambda i,d=d: int(d)*i)
            a = types.get(left)
            for opt in ty:
                a = opt(a)
            fields.append((b,a))
    return name, fields

In [363]:
parse_struct("""
typdef struct ts_vertex {
 struct ts_vertex* q;
 int d;
 int x[3],y, k[9][2];
 char *tape;

} ts_vertex;
""",ts_types)

('ts_vertex',
 [('q', __main__.LP_ts_vertex),
  ('d', ctypes.c_int),
  ('x', __main__.c_int_Array_3),
  ('y', ctypes.c_int),
  ('k', __main__.c_int_Array_2_Array_9),
  ('tape', ctypes.LP_c_char)])

In [365]:
structs = {}
for item in items:
    if item[0]=='struct':
        name, fields = parse_struct('\n'.join(item[1]),ts_types)
        if not ts_types[name]._fields_:
            ts_types[name]._fields_.extend(fields)
        if fields:
            structs[name]=fields
structs

{'ts_coord': [('e1', ctypes.c_double),
  ('e2', ctypes.c_double),
  ('e3', ctypes.c_double),
  ('coord_type', ctypes.c_uint)],
 'ts_vertex': [('x', ctypes.c_double),
  ('y', ctypes.c_double),
  ('z', ctypes.c_double),
  ('mean_curvature', ctypes.c_double),
  ('gaussian_curvature', ctypes.c_double),
  ('mean_energy', ctypes.c_double),
  ('gaussian_energy', ctypes.c_double),
  ('energy', ctypes.c_double),
  ('xk', ctypes.c_double),
  ('xk2', ctypes.c_double),
  ('w', ctypes.c_double),
  ('c', ctypes.c_double),
  ('nx', ctypes.c_double),
  ('ny', ctypes.c_double),
  ('nz', ctypes.c_double),
  ('nx2', ctypes.c_double),
  ('ny2', ctypes.c_double),
  ('nz2', ctypes.c_double),
  ('f', ctypes.c_double),
  ('fx', ctypes.c_double),
  ('fy', ctypes.c_double),
  ('fz', ctypes.c_double),
  ('ad_w', ctypes.c_double),
  ('d', ctypes.c_double),
  ('dx', ctypes.c_double),
  ('dy', ctypes.c_double),
  ('dz', ctypes.c_double),
  ('eig0', ts_wrapper.c_double_Array_3),
  ('eig1', ts_wrapper.c_double_Array_

In [356]:
types = ts_types
parse_struct(['ts_double bond_length;',
   'ts_double energy;',
   'ts_double x,y,z;',
   'ts_vertex *vtx1;',
   'ts_vertex *vtx2;',
   'ts_idx idx;',],types)

[('energy', ctypes.c_double),
 ('x', ctypes.c_double),
 ('y', ctypes.c_double),
 ('z', ctypes.c_double),
 ('vtx1', __main__.LP_ts_vertex),
 ('vtx2', __main__.LP_ts_vertex)]

In [134]:
ts_types['ts_vertex']._fields_

[('x', ctypes.c_double),
 ('y', ctypes.c_double),
 ('z', ctypes.c_double),
 ('mean_curvature', ctypes.c_double),
 ('gaussian_curvature', ctypes.c_double),
 ('mean_energy', ctypes.c_double),
 ('gaussian_energy', ctypes.c_double),
 ('energy', ctypes.c_double),
 ('xk', ctypes.c_double),
 ('xk2', ctypes.c_double),
 ('w', ctypes.c_double),
 ('c', ctypes.c_double),
 ('nx', ctypes.c_double),
 ('ny', ctypes.c_double),
 ('nz', ctypes.c_double),
 ('nx2', ctypes.c_double),
 ('ny2', ctypes.c_double),
 ('nz2', ctypes.c_double),
 ('f', ctypes.c_double),
 ('fx', ctypes.c_double),
 ('fy', ctypes.c_double),
 ('fz', ctypes.c_double),
 ('ad_w', ctypes.c_double),
 ('d', ctypes.c_double),
 ('dx', ctypes.c_double),
 ('dy', ctypes.c_double),
 ('dz', ctypes.c_double),
 ('eig0', ts_wrapper.c_double_Array_3),
 ('eig1', ts_wrapper.c_double_Array_3),
 ('eig2', ts_wrapper.c_double_Array_3),
 ('eig_v0', ctypes.c_double),
 ('eig_v1', ctypes.c_double),
 ('eig_v2', ctypes.c_double),
 ('mean_curvature2', ctypes.c_doubl

In [ ]:
#for i in range(n):
#    f=open('cluster_dist_passive_E_2_pt_5_c_350.dat','a')
#    f.write('{}, {} \n'.format(i+1,cl_array[i]/icount))